# 6. Most common operons

Now that we've richly annotated 5 the 5-gene radius surrounding rhodopsins, we want to know...<br>
What annotations are the most common in these flanking genes?<br>
In order to uncover operons or gene blocks<Br>
Which could illuminate the functional context of each rhodopsin

In [2]:
import pandas as pd
import numpy as np
from os.path import exists
import ast
import sys
import itertools
import collections
from Bio import SeqIO

MODE_verbose = False # Make 'True' to see report in Part I
MODE_printIDS = False  #Super-verbose: Prints the IDs of each contig containing rhodopsin

LIST_cols_most_to_least_common = ["3' 2Fe-2S binding dom.", "3' flotillin", "5' AMP-binding enz.", "5' VirC1", "5' amidohydrolase fam.", "5' A.A. permease", "5' periplasmic-binding prot.", "3' CrtE", "3' L-rhamnose isomerase", "5' brp/blh", "3' tRNA synthetase I", "3' haemolysin-III related", "5' fasciclin", "3' fatty acid desaturase", "3' brp/blh", "5' thiamine pyrophosphate", "5' TMEM205-like", "5' AI-2E fam. transporter", "5' toxin-antitoxin", "3' phosphotransferase fam.", "3' SOUL heme-binding prot.", "3' ABC1 kinase-like", "5' FGGY fam. carb. kinases"]

PATH_out = "./output/rhodopsin_operon_presence_absence_Dec11.csv"

PATH_in = "../5_reannotate_rhodopsin_neighborhood/output/B_rhodopsin_neighborhood.csv"
#PATH_in = './input/rhodopsin_neighborhood.csv' # w/ foldseek annots

# Need Dram table to get strand info for the rhodopsins
PATH_dram = "../0_get_DRAM_AAs_for_SAGs_w_rhodopsins/dram_926_SAGs_combined.csv"
#PATH_dram = '/mnt/scgc/stepanauskas_nfs/projects/gorg-dark/dram/dram_annot_251010.csv'

PATH_temp = './temp/rhodopsin_neighborhood_plus_strandedness.csv' # Saves time re-loading Dram

LIST_key_order = ['annot_multisrc of gene -5','annot_multisrc of gene -4','annot_multisrc of gene -3','annot_multisrc of gene -2','annot_multisrc of gene -1','annot_multisrc of gene +1','annot_multisrc of gene +2','annot_multisrc of gene +3','annot_multisrc of gene +4','annot_multisrc of gene +5']

#### Top 20 annotations UPSTREAM of rhodopsin (at pos -1) and their frequency (manually tabulated)
## Note that there are fewer than 20 because many annotations were variations of the same thing (i.e. reworded but sharing a Pfam)
#1 AMP-binding enzyme [PF00501.31]; Acetyl-coenzyme A synthetase N-terminus [PF16177.8]; AMP-binding enzyme C-terminal domain [PF13193.9]   280
#2 Unannotated    165 (Can't search for this)
#3 "VirC1" protein 230
#4 Amidohydrolase family [PF01979.23]; Amidohydrolase family [PF07969.14] 89
#5 Branched-chain amino acid transport system / permease component [PF02653.19]   59
#6 Periplasmic binding protein domain [PF13407.9]; Periplasmic binding proteins and sugar binding domain of LacI family [PF00532.24]  41
#7 AMP-binding enzyme [PF00501.31] 11
#8 Beta-carotene 15,15'-dioxygenase [PF15461.9]|brp/blh 
#9 Fasciclin domain [PF02469.25] 
#10 Thiamine pyrophosphate enzyme, N-terminal TPP binding domain [PF02776.21] 8
#11 TMEM205-like domain-containing protein  8
#12 AI-2E family transporter [PF01594.19]    8
#13 SUKH-4 immunity protein of toxin-antitoxin system       6
#14 FGGY family of carbohydrate kinases, N-terminal domain [PF00370.24]    5
 
#### Distilled to their core unique patterns (Pfam if possible, but substring if pfam is lacking or if substring is more prevalent)
DICT_upstream_annotPattern2Nickname={
"PF00501":"5' AMP-binding enz.",
"VirC1":"5' VirC1",
"PF01979":"5' amidohydrolase fam.",
"PF02653":"5' A.A. permease",
"PF13407":"5' periplasmic-binding prot.",
"blh":"5' brp/blh",
"PF02469":"5' fasciclin",
"PF02776":"5' thiamine pyrophosphate",
"TMEM205":"5' TMEM205-like",
"PF01594":"5' AI-2E fam. transporter",
"toxin":"5' toxin-antitoxin",
"PF00370":"5' FGGY fam. carb. kinases"}

#### Top 20 patterns DOWNSTREAM (at pos +1) and their frequency (manually tabulated)
## Note that there are fewer than 20 because many annotations were variations of teh same thing (i.e. sharing a Pfam)
#1 2Fe-2S iron-sulfur cluster binding domain [PF00111.30]   422
#2 flotillin    384
#3 Unannotated    83 (Can't search for this)
#4 Polyprenyl synthetase [PF00348.20]|CrtE    25
#5 L-rhamnose isomerase (RhaA) [PF06134.14]   24
#6 Fatty acid desaturase [PF00487.27] 9 
#7  Beta-carotene 15,15'-dioxygenase [PF15461.9]|brp/blh   7
#8 tRNA synthetases class I (R) [PF00750.22]; DALR anticodon binding domain [PF05746.18]   7
#9 Phosphotransferase enzyme family [PF01636.26]; Choline/ethanolamine kinase [PF01633.23]; Ecdysteroid kinase-like family [PF02958.23]  6
#10 SOUL heme-binding protein [PF04832.15]                                                                  6
#11 ABC1 atypical kinase-like domain [PF03109.19]; Cytochrome P450 [PF00067.25]; Protein kinase domain [PF00069.28]                   6
#12 Haemolysin-III related [PF03006.23]                           5

#### Distilled to their core unique patterns (Pfam if possible, but substring if pfam is lacking or if substring is more prevalent)
DICT_downstream_annotPattern2Nickname={
"PF00111":"3' 2Fe-2S binding dom.",
"lotillin":"3' flotillin",
"CrtE":"3' CrtE",
"PF06134":"3' L-rhamnose isomerase",
"PF00487":"3' fatty acid desaturase",
"blh":"3' brp/blh",
"PF00750":"3' tRNA synthetase I",
"PF01636":"3' phosphotransferase fam.",
"PF04832":"3' SOUL heme-binding prot.",
"PF03109":"3' ABC1 kinase-like",
"PF03006":"3' haemolysin-III related"
}


def I_string_dict_to_dictionary(STR_of_dict):
    '''
    For parseability, we want each dictlike entry to have 10 keys like:
    ['annot_multisrc of gene -5','annot_multisrc of gene -4','annot_multisrc of gene -3','annot_multisrc of gene -2','annot_multisrc of gene -1','annot_multisrc of gene +1','annot_multisrc of gene +2','annot_multisrc of gene +3','annot_multisrc of gene +4','annot_multisrc of gene +5']
    But if the contig was too short, some keys are missing.
    So create those missing keys...
    and give them vals that are empty strings.
    '''
    # Add missing positions/keys if necessary
    DICT_hits = ast.literal_eval(STR_of_dict)
    for key in LIST_key_order:
        if key not in DICT_hits.keys():
            DICT_hits[key]=''
            
    # reorder dictionary so stays in LIST_key_order
    orderedDICT_hits = collections.OrderedDict((k, DICT_hits[k]) for k in LIST_key_order)
    DICT_hits = dict(orderedDICT_hits)
    
    return DICT_hits

def II_dict_to_tilda_separated_string(DICT):
    '''
    '''
    STR_from_dict = ""
    for key in ['annot_multisrc of gene -5','annot_multisrc of gene -4','annot_multisrc of gene -3','annot_multisrc of gene -2','annot_multisrc of gene -1','annot_multisrc of gene +1','annot_multisrc of gene +2','annot_multisrc of gene +3','annot_multisrc of gene +4','annot_multisrc of gene +5']:
        LIST_Pfam_hits = DICT[key]
        #Drop Pfam hits that are just "None"
        LIST_Pfam_hits = [i for i in LIST_Pfam_hits if i != 'None'] 
        #Drop Pfam hits that are just numbers
        LIST_Pfam_hits = [i for i in LIST_Pfam_hits if type(i) == str]
        if len(LIST_Pfam_hits) == 0:
            LIST_Pfam_hits = ''
        val = '|'.join(LIST_Pfam_hits)
        STR_from_dict = STR_from_dict+val+"~"
    STR_from_dict = STR_from_dict[:-1] # Drop trailing tilda
    return STR_from_dict

def III_list_annots_up_or_downstream_of_rhodopsin_starting_at_specific_pattern(DF_in, desired_substring_in_annotation, up_OR_down, MODE_annots_or_occupancy_or_rhodopsinIDs):
    '''
    If MODE_annots_or_occupancy_or_rhodopsinIDs == 'annots':
    E.g. We've seen that "flotillin" is enriched near rhodopsin, so...
    This function would...
        Subset the DF so we're only looking at neighborhods where substring "flotillin" is found in the annotation next to rhodopsin
        If set to upstream ("up"):
            Requires "flotillin" at -1, then reruns a list of upstream annots (-5,-4,-3,-2,-1)
        If set to downstream ("down"):
            Requires "flotillin" at +1, then returns a list of downstream annots (+1,+2,+3,+4,+5)
    
    E.g. returns: ['flotillin', 'carboxyblahblah PF230303','','DUF224','actin PF24545']
    
    INSTEAD: if MODE_annots_or_occupancy_or_rhodopsinIDs == 'occupancy':
    returns a list of the frequency of that annot at each site:
    
    E.g. returns [1.0, 0.09, 0.3, 0.1 0.0 ]
    
    INSTEAD: if MODE_annots_or_occupancy_or_rhodopsinIDs == 'rhodopsinIDs'
    returns a list of all rhodopsins with that pattern down/upstream (i.e. all rhodopsins with that putative operon-type)
    '''
        
    LIST_annots = []
    LIST_occupancy = []
    
    if up_OR_down=='up':
        LIST_coords = ["-5","-4","-3","-2","-1"]
        _DF = DF_in.loc[DF_in["-1"].str.contains(desired_substring_in_annotation)]
        #LIST_annots=[_DF[i].value_counts()[0] for i in LIST_coords]
        
    elif up_OR_down=='down':
        LIST_coords = ["+1","+2","+3","+4","+5"]
        _DF = DF_in.loc[DF_in["+1"].str.contains(desired_substring_in_annotation)]
        #LIST_annots=[_DF[i].value_counts()[0] for i in LIST_coords]
    else:
        sys.exit("Error. You must specify 'up' or 'down'!")
    
    LIST_rhodopsin_ids = _DF['rhodopsin_gene'].to_list()
    
    for pos in LIST_coords:
        DICT_annots_at_this_position = _DF[pos].value_counts()
        top_annot = list(DICT_annots_at_this_position.keys())[0]
        
        # How many times is that annotation pattern at that position (e.g. 'PF00501' at position +1)
        num_this_annot = DICT_annots_at_this_position[top_annot]
        # Occupancy rate = perc of proteins at that position with that pattern (e.g. 0.5, a.k.a. 50% at pos +1 contain 'PF00501')
        rate_occupancy = round(num_this_annot / len(_DF),2)
       
        LIST_annots.append(top_annot)
        LIST_occupancy.append(rate_occupancy)
        
    if MODE_annots_or_occupancy_or_rhodopsinIDs == 'annots':
        return LIST_annots
    elif MODE_annots_or_occupancy_or_rhodopsinIDs == 'occupancy':
        return LIST_occupancy
    elif MODE_annots_or_occupancy_or_rhodopsinIDs == 'rhodopsinIDs':
        return LIST_rhodopsin_ids

### Read in Tianyi's neighborhood table
DF_raw = pd.read_csv(PATH_in)

### Still needs strand info, for rhodopsins, so retrieve that from Dram
if not exists(PATH_temp):
    print("Getting strandedness of rhodopsin from the massive Dram table")
    DF_dram = pd.read_csv(PATH_dram)
    DF_dram = DF_dram[['...1','strandedness']]
    DF_dram = DF_dram.rename(columns={'...1':'rhodopsin_gene'})
    DF_merged = DF_raw.merge(DF_dram, on='rhodopsin_gene', how='left')
    DF_merged = DF_merged.rename(columns={'strandedness':'rhodopsin_strandedness'})
    DF_merged.to_csv(PATH_temp, index=False)
    print("done")
    
print("Loading gene neighborhood table to get flanking gene info")
DF1 = pd.read_csv(PATH_temp, index_col=None)


### Preformat 'flanking genes' column before I can split it into 10 columns 
DF1['TMP_dict'] = DF1['flanking_genes'].apply(I_string_dict_to_dictionary)
DF1['TMP_tildasep'] = DF1['TMP_dict'].apply(II_dict_to_tilda_separated_string)

print("Splitting flanking gene info into 10 columns.")
### Split 'flanking genes' into 10 gene neighborhood columns, by coordinate
DF1[['-5','-4','-3','-2','-1','+1','+2','+3','+4','+5']] = DF1['TMP_tildasep'].str.split('~',expand=True)

### Drop temp columns
DF1 = DF1.drop(columns=['TMP_dict','TMP_tildasep'])

### For rows where rhodopsin is on negative strand (-1), flip the order of the flanking genes
# Requires splitting the table temporarily

print('\nOf '+str(len(DF1))+' contigs with rhodopsin...')
DF1forward = DF1.loc[DF1['rhodopsin_strandedness']==1]
print(str(len(DF1forward))+" rhodopsins are on the forward/positive strand")

DF1reverse = DF1.loc[DF1['rhodopsin_strandedness']==-1]
print(str(len(DF1reverse))+" rhodopsins are on the reverse/negative strand")

DF1_rev_fixed = DF1reverse
DF1_rev_fixed = DF1_rev_fixed.drop(columns=['-5','-4','-3','-2','-1','+1','+2','+3','+4','+5'])
DF1_rev_fixed['-5'] = DF1reverse['+5']
DF1_rev_fixed['-4'] = DF1reverse['+4']
DF1_rev_fixed['-3'] = DF1reverse['+3']
DF1_rev_fixed['-2'] = DF1reverse['+2']
DF1_rev_fixed['-1'] = DF1reverse['+1']
DF1_rev_fixed['+1'] = DF1reverse['-1']
DF1_rev_fixed['+2'] = DF1reverse['-2']
DF1_rev_fixed['+3'] = DF1reverse['-3']
DF1_rev_fixed['+4'] = DF1reverse['-4']
DF1_rev_fixed['+5'] = DF1reverse['-5']

print("For those negative-stranded rhodopsins, renumbered the flanking genes so that neg positions become positive and vice versa")
print("In other words, flanking genes are numbered -5 to +5 with respect to the 5'->3' orientation or rhodopsin.")

DF2 = pd.concat([DF1forward, DF1_rev_fixed])
DF2 = DF2.sort_values(by=['rhodopsin_gene'])
print(str(len(DF2))+" contigs following re-orientation")


########### PART I. DISPLAY MOST COMMON NEIGHBORING ANNOTATIONS (for exploratory purposes, i.e. to discover operons) ##################

count_operonic_rhodopsins = 0

if MODE_verbose==True:
    print('\nSearching for potential operons, by walking outward from most common neighbors to rhodopsin.')
    
    print("\n\nREPORTING UPSTREAM GENE NEIGHBORHOOD")
    count=0
    #for up_pattern in LIST_search_upstream:
    for up_pattern in list(DICT_upstream_annotPattern2Nickname.keys()):
        count+=1
        _LIST_annots = III_list_annots_up_or_downstream_of_rhodopsin_starting_at_specific_pattern(DF2, up_pattern, 'up', 'annots')

        _LIST_occupancy_per_annot = III_list_annots_up_or_downstream_of_rhodopsin_starting_at_specific_pattern(DF2, up_pattern, 'up', 'occupancy')
        _LIST_occupancy_per_annot = [str(i) for i in _LIST_occupancy_per_annot]
    
        # All these operons ended with rhodopsin
        _LIST_annots.append('RHODOPSIN')
        _LIST_occupancy_per_annot.append('1.0')
    
        count_operonic_rhodopsins = count_operonic_rhodopsins+len(DF2.loc[DF2['-1'].str.contains(up_pattern)])
    
        print("\n\n"+str(len(DF2.loc[DF2['-1'].str.contains(up_pattern)]))+' of neighborhoods start from this enriched annotation/substring in the upstream neighbor of rhodopsin: "'+ up_pattern+'"')
        print("with annotations in this order  (1 gene per line):\n" + '\n*'.join(_LIST_annots))
        print("\nWhere X% of each position matches the annotations above:\n"+','.join(_LIST_occupancy_per_annot))

        if MODE_printIDS==True:
            print("\nThe following rhodopsins have this neighboring annot:\n")
            print(",".join(III_list_annots_up_or_downstream_of_rhodopsin_starting_at_specific_pattern(DF2, up_pattern, 'up', 'rhodopsinIDs')))
        
if MODE_verbose==True:
    print("\n\nREPORTING DOWNSTREAM GENE NEIGHBORHOOD")
    count=0
    #for down_pattern in LIST_search_downstream:
    for down_pattern in list(DICT_downstream_annotPattern2Nickname.keys()):
        count+=1
        _LIST_annots = III_list_annots_up_or_downstream_of_rhodopsin_starting_at_specific_pattern(DF2, down_pattern, 'down', 'annots')
    
        _LIST_occupancy_per_annot = III_list_annots_up_or_downstream_of_rhodopsin_starting_at_specific_pattern(DF2, down_pattern, 'down', 'occupancy')
        _LIST_occupancy_per_annot = [str(i) for i in _LIST_occupancy_per_annot]
    
        # All these operons started with rhodopsin
        _LIST_annots = ['RHODOPSIN']+_LIST_annots
        _LIST_occupancy_per_annot = ['1.0']+_LIST_occupancy_per_annot
    
        count_operonic_rhodopsins = count_operonic_rhodopsins+len(DF2.loc[DF2['-1'].str.contains(up_pattern)])
    
        print("\n\n"+str(len(DF2.loc[DF2['+1'].str.contains(down_pattern)]))+' of neighborhoods start from this enriched annotation/substring in the downstream neighbor of rhodopsin: "'+ down_pattern+'"')
        print("with annotations in this order (1 gene per line):\n" + '\n*'.join(_LIST_annots))
        print("\n Where X% of each position matches the annotations above:\n"+','.join(_LIST_occupancy_per_annot))

        if MODE_printIDS==True:
            print("\nThe following rhodopsins have this neighboring annot:\n")
            print(",".join(III_list_annots_up_or_downstream_of_rhodopsin_starting_at_specific_pattern(DF2, down_pattern, 'down', 'rhodopsinIDs')))


########### PART II. TABULATE PRESENCE/ABSENCE FOR MOST COMMON ANNOTS FLANKING RHODOPSIN ##################

LIST_DoesPatternFlankRhodopsin = []  #For each pattern, contains  a DF of binary vals for presence or absence of that pattern v. each rhodopsin SeqID
# Basically, contains 23 DFs, since there are 23 patterns.
# And each DF has a row for each rhodopsin ID, and the cells can be valued 0 (absent) or 1 (present) 

print("\n\n~~~~~~~ Annotations Flanking Downstream ~~~~~~~~~\n")
for down_pattern in list(DICT_downstream_annotPattern2Nickname.keys()):
    _pattern_name = DICT_downstream_annotPattern2Nickname[down_pattern]
    # List of rhodIDs to check for pattern
    _LIST = DF2.loc[DF2['+1'].str.contains(down_pattern)]['rhodopsin_gene'].to_list()
    _DF = pd.Series(data=np.nan, index=DF2['rhodopsin_gene'], name=_pattern_name).to_frame()
    _DF[_pattern_name] = _DF.index.isin(_LIST).astype(int)
    LIST_DoesPatternFlankRhodopsin.append(_DF)
    print(_pattern_name, "~", down_pattern, "~", str(_DF[DICT_downstream_annotPattern2Nickname[down_pattern]].sum()))

print("\n\n~~~~~~~~~ Annotations Flanking Upstream ~~~~~~~~~~~\n")
for up_pattern in list(DICT_upstream_annotPattern2Nickname.keys()):
    _pattern_name= DICT_upstream_annotPattern2Nickname[up_pattern]
    # List of rhodIDs to check for pattern
    _LIST = DF2.loc[DF2['-1'].str.contains(up_pattern)]['rhodopsin_gene'].to_list()
    _DF = pd.Series(data=np.nan, index=DF2['rhodopsin_gene'], name=_pattern_name).to_frame()
    _DF[DICT_upstream_annotPattern2Nickname[up_pattern]] = _DF.index.isin(_LIST).astype(int)
    LIST_DoesPatternFlankRhodopsin.append(_DF)
    print(_pattern_name, "~", up_pattern, "~", str(_DF[_pattern_name].sum()))
    
#### Save info to a Presence/Absence table (each row is a rhodopsin ID from GORG-Dark. Each cell is whether that rhodopsin is flanked by that annotation or not)

DF_presence_absence = pd.concat(LIST_DoesPatternFlankRhodopsin, axis=1)
DF_presence_absence = DF_presence_absence[LIST_cols_most_to_least_common]
DF_presence_absence.to_csv(PATH_out)
print("\nSaved results to Presence/Absence table.\nUse the table to the plot presence/absence of flanking genes on iToL phylogeny")

Getting strandedness of rhodopsin from the massive Dram table
done
Loading gene neighborhood table to get flanking gene info
Splitting flanking gene info into 10 columns.

Of 1095 contigs with rhodopsin...
572 rhodopsins are on the forward/positive strand
523 rhodopsins are on the reverse/negative strand
For those negative-stranded rhodopsins, renumbered the flanking genes so that neg positions become positive and vice versa
In other words, flanking genes are numbered -5 to +5 with respect to the 5'->3' orientation or rhodopsin.
1095 contigs following re-orientation


~~~~~~~ Annotations Flanking Downstream ~~~~~~~~~

3' 2Fe-2S binding dom. ~ PF00111 ~ 422
3' flotillin ~ lotillin ~ 391
3' CrtE ~ CrtE ~ 25
3' L-rhamnose isomerase ~ PF06134 ~ 24
3' fatty acid desaturase ~ PF00487 ~ 9
3' brp/blh ~ blh ~ 8
3' tRNA synthetase I ~ PF00750 ~ 9
3' phosphotransferase fam. ~ PF01636 ~ 6
3' SOUL heme-binding prot. ~ PF04832 ~ 6
3' ABC1 kinase-like ~ PF03109 ~ 6
3' haemolysin-III related ~ PF03006